In [6]:
import cv2
import numpy as np

points = []

def get_points(event, x, y, flags, param):
    global points
    img = param["img"]

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append([x, y])
        cv2.circle(img, (x, y), 6, (0, 255, 0), -1)
        cv2.imshow("Calibracion", img)
        print(points)

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

clone = frame.copy()
cv2.imshow("Calibracion", clone)
cv2.setMouseCallback("Calibracion", get_points, {"img": clone})
cv2.waitKey(0)
cv2.destroyAllWindows()

points = np.array(points, dtype=np.float32)

np.save("piano_points.npy", points)
print("Puntos guardados:", points)

[[149, 196]]
[[149, 196], [527, 183]]
[[149, 196], [527, 183], [576, 403]]
[[149, 196], [527, 183], [576, 403], [138, 424]]
Puntos guardados: [[149. 196.]
 [527. 183.]
 [576. 403.]
 [138. 424.]]


In [ ]:
import pygame
import cv2
import numpy as np
import mediapipe as mp

SAMPLE_RATE = 44100

def envelope_adsr(t, duration):
    """
    Envolvente aproximada tipo piano:
    - Ataque muy rápido
    - Decaimiento corto
    - Sustain medio
    - Release al final
    """
    attack = 0.01       # 10 ms
    decay = 0.08        # 80 ms
    sustain = 0.6
    release = 0.3       # 300 ms

    env = np.zeros_like(t)

    # Attack
    idx = t < attack
    env[idx] = (t[idx] / attack)

    # Decay
    idx = (t >= attack) & (t < attack + decay)
    env[idx] = 1 - (1 - sustain) * ((t[idx] - attack) / decay)

    # Sustain
    idx = (t >= attack + decay) & (t < duration - release)
    env[idx] = sustain

    # Release
    idx = t >= (duration - release)
    if np.any(idx):
        env[idx] = sustain * (1 - (t[idx] - (duration - release)) / release)
        env[env < 0] = 0.0

    return env


def generate_piano_tone(frequency, duration=0.8, volume=0.5, sample_rate=SAMPLE_RATE):
    """
    Genera una nota tipo piano:
    - Fundamental ligeramente desafinada en 2 cuerdas (detune)
    - Armónicos extra
    - Envolvente ADSR tipo piano
    - Resonancia simple de caja
    """
    t = np.linspace(0, duration, int(sample_rate * duration), False)

    # Envolvente
    env = envelope_adsr(t, duration)

    # Detune pequeño (2 cuerdas por nota)
    f1 = frequency * 0.997
    f2 = frequency * 1.003
    base = 0.5 * (np.sin(2 * np.pi * f1 * t) + np.sin(2 * np.pi * f2 * t))

    # Armónicos (sobre la fundamental "ideal")
    wave = (
        1.0 * base +
        0.6 * np.sin(2 * np.pi * frequency * 2 * t) +
        0.3 * np.sin(2 * np.pi * frequency * 3 * t) +
        0.15 * np.sin(2 * np.pi * frequency * 4 * t) +
        0.10 * np.sin(2 * np.pi * frequency * 5 * t)
    )

    # Aplicar envolvente
    wave *= env

    # Resonancia simple (filtro IIR muy básico)
    r = 0.4
    y = np.zeros_like(wave)
    for i in range(len(wave)):
        if i == 0:
            y[i] = wave[i]
        else:
            y[i] = wave[i] + r * y[i - 1]

    # Normalizar
    max_val = np.max(np.abs(y))
    if max_val > 0:
        y = y / max_val

    y = (y * 32767 * volume).astype(np.int16)
    return y


# Frecuencias de la escala de DO mayor
frequencies = {
    "DO": 261.63,
    "RE": 293.66,
    "MI": 329.63,
    "FA": 349.23,
    "SOL": 392.00,
    "LA": 440.00,
    "SI": 493.88,
    "DO2": 523.25
}

# Inicializar mixer en ESTÉREO
pygame.mixer.init(frequency=SAMPLE_RATE, size=-16, channels=2)

NOTES = ["DO", "RE", "MI", "FA", "SOL", "LA", "SI", "DO2"]

note_sounds = []
for note in NOTES:
    # Generar la nota tipo piano
    wave = generate_piano_tone(frequencies[note], duration=0.8, volume=0.7)

    # Pasar a estéreo duplicando el canal (2D para pygame)
    wave_stereo = np.column_stack((wave, wave))

    sound = pygame.sndarray.make_sound(wave_stereo)
    note_sounds.append(sound)

c:\Users\Usuario\anaconda3\envs\VC_Final\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'np' is not defined

In [ ]:


# ---------- CONFIGURACIÓN ----------
touch_threshold = -0.035
FINGERS = [4, 8, 12, 16, 20]

prev_fy = {f: None for f in FINGERS}
finger_was_down = {f: False for f in FINGERS}
fz_smooth = {f: None for f in FINGERS}
finger_velocity = {f: 0 for f in FINGERS}

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

img_size = (800, 300)

src_pts = np.load("piano_points.npy")

dst_pts = np.array([
    [0, 0],
    [img_size[0], 0],
    [img_size[0], img_size[1]],
    [0, img_size[1]]
], dtype=np.float32)

H = cv2.getPerspectiveTransform(src_pts, dst_pts)

cap = cv2.VideoCapture(0)

# ---------- BUCLE ----------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    h_img, w_img, _ = frame.shape
    piano_flat = cv2.warpPerspective(frame, H, img_size)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    finger_points = {}
    finger_z = {}

    # ---------- DETECTAR TODOS LOS DEDOS ----------
    if result.multi_hand_landmarks:
        hand_lms = result.multi_hand_landmarks[0]

        for f_id in FINGERS:
            lm = hand_lms.landmark[f_id]

            fx = int(lm.x * w_img)
            fy = int(lm.y * h_img)
            fz = lm.z

            # Suavizado Z individual
            if fz_smooth[f_id] is None:
                fz_smooth[f_id] = fz
            else:
                fz_smooth[f_id] = 0.7 * fz_smooth[f_id] + 0.3 * fz

            fz = fz_smooth[f_id]
            finger_z[f_id] = fz

            cv2.circle(frame, (fx, fy), 8, (0, 255, 255), -1)

            # Transformación de dedo al piano
            p = np.array([[[fx, fy]]], dtype=np.float32)
            p_flat = cv2.perspectiveTransform(p, H)

            fx_p = int(p_flat[0][0][0])
            fy_p = int(p_flat[0][0][1])

            finger_points[f_id] = (fx_p, fy_p)

    # ---------- DIBUJAR TECLAS UNA SOLA VEZ ----------
    key_width = img_size[0] // 8
    for i in range(8):
        x1 = i * key_width
        x2 = x1 + key_width
        cv2.rectangle(piano_flat, (x1, 0), (x2, img_size[1]), (255, 0, 0), 2)
        cv2.putText(piano_flat, NOTES[i], (x1+20, img_size[1]-30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    # ---------- DETECCIÓN DE TOQUES ----------
    pressed_keys = set()

    for f_id in FINGERS:
        if f_id not in finger_points:
            continue

        fx_p, fy_p = finger_points[f_id]
        fz = finger_z[f_id]

        # VELOCIDAD DE BAJADA
        if prev_fy[f_id] is not None:
            velocity = fy_p - prev_fy[f_id]
            finger_velocity[f_id] = velocity
        else:
            velocity = 0

        prev_fy[f_id] = fy_p

        finger_is_down = (fz < touch_threshold and velocity > 6)

        # DETECCIÓN DE TECLA
        for i in range(8):
            x1 = i * key_width
            x2 = x1 + key_width

            inside = (x1 < fx_p < x2 and 0 < fy_p < img_size[1])

            if inside and finger_is_down and not finger_was_down[f_id]:
                pressed_keys.add((f_id, i))  # dedo + tecla

        finger_was_down[f_id] = finger_is_down

    # ---------- REPRODUCIR NOTAS ----------
    for f_id, key in pressed_keys:
        vel = max(0, min(finger_velocity[f_id], 20))
        vol = 0.2 + 0.8 * (vel / 20)

        note_sounds[key].set_volume(vol)
        note_sounds[key].play()

    # ---------- VENTANAS ----------
    cv2.imshow("Vista Camara", frame)
    cv2.imshow("Piano Plano", piano_flat)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
pygame.quit()


error: mixer not initialized

: 